## Environment Set-Up

It seems first we need to define where the grid scenario data lives and make it known to our Python session.

In [12]:
model_dir = 'agents/'
# input_dir = '../chronics_contestants/'  
# output_dir = 'codalab_tools/output/'
parameters_path = '../chronics_contestants/chronics/0000'  # Unzipped the public data in same dir as repo dir
game_level_path = '../chronics_contestants/'


from sys import path
path.append(model_dir)
# path.append(input_dir)
# path.append(output_dir)


The file names are all standardized for each of the scenarios provided (aka "chronics")

In [4]:
##
# Set file names for loading scenarios
# These are the files in each of the four-digit "chronics" folders
# Essentially they've provided a bunch of scenarios to train on
##
loads_p_file = '_N_loads_p.csv' #active power chronics for loads
prods_p_file = '_N_prods_p.csv'  #active power chronics for productions
datetimes_file = '_N_datetimes.csv' #timesstamps of the chronics
maintenance_file = 'maintenance.csv' #maintenance operation chronics. No maintenance considered in the first challenge
hazards_file = 'hazards.csv'   #harzard chronics that disconnect lines. No hazards considered in the first challenge
imaps_file = '_N_imaps.csv' #thermal limits of the lines

Load all third party and project libraries

In [6]:
# load third-party libs from example notebook
import sys
import logging
import os
import warnings
import numpy as np
import pandas as pd
import utils.scoring_program.evaluate  # Based upon where this is in our repo
import matplotlib.pyplot as plt
import json

warnings.simplefilter(action='ignore', category=FutureWarning)

# Additional third-party libs for q-learning
import random

from utils.ingestion_program.runner import Runner  # Based upon where this is in our repo
import pypownet.environment

## Initialize the Environment

Use the function provided in one of the example notebooks to init the environment with from the chronic file.

In [15]:
# Function from Run and Submit agent notebook
def set_environment(game_level = "datasets", start_id=0):
    """
        Load the first chronic (scenario) in the directory public_data/datasets
    """
    return pypownet.environment.RunEnv(parameters_folder=os.path.abspath(parameters_path),
                                              game_level=os.path.abspath(game_level_path),
                                              chronic_looping_mode='natural', start_id=start_id,
                                              game_over_mode="soft")

environment = set_environment()

/!\ Using default reward signal, as reward_signal.py file is not found


## Explore the Environment

In [20]:
# Look at the load ids
load_ids = environment.action_space.loads_subs_ids
print ('Num of loads in total: {} -> Load ids:'.format(len(load_ids), load_ids))
print (load_ids)

# Look at the generator ids
gen_id = environment.action_space.prods_subs_ids
print ('Num of gen in total: {} -> Generators id:'.format(len(gen_id), gen_id))
print (gen_id)


Num of loads in total: 11 -> Load ids:
[ 2  3  4  5  6  9 10 11 12 13 14]
Num of gen in total: 5 -> Generators id:
[1 2 3 6 8]


## A Feeble Attempt to Set-Up Q Learning 

In [122]:
# # Init arbitrary values

# # Create observation_space and action_space
# action_space = environment.action_space
# observation_space = environment.observation_space
# game = environment.game  # I have no clue what this is doing

# # Create a do_nothing action (all values in action_space set to zero)
# action_do_nothing = action_space.get_do_nothing_action()  # This is a vector of length 76

# # Run one step to get observations
# obs, *_ = environment.step(action_do_nothing)
# #print(type(obs))

# environment.reward_signal.compute_reward(obs, action_do_nothing, 0)
# # obs.shape  # This is just a vector of length 538

# test = game.export_observation().as_ac_minimalist()

# test.as_array().shape
# pypownet.environment.TooManyProductionsCut.mro
#reward_signal = game.get_reward_signal_class
#reward_signal.compute_reward(game.export_observation(), action_space.get_do_nothing_action(), flag = 1)
# game.export_observation().thermal_limits
# Create observation_space and action_space
action_space = environment.action_space
observation_space = environment.observation_space
game = environment.game  # TODO(nadir.sidi): Review Game class

# Create a do_nothing action (all values in action_space set to zero)
action_do_nothing = action_space.get_do_nothing_action()  # This is a vector of length 76

# Run one step to get observations
# obs, *_ = environment.step(action_do_nothing)
# print(type(obs))
# obs.shape  # This is just a vector of length 538

# Get the Observation class from the game to get the minimalist ac observations
obs_min = game.export_observation().as_ac_minimalist().as_array()
obs_min.astype(int)




array([  17,   76,   37,    6,    8,   22,    7,    2,    4,   10,   12,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         17,   75,   37,    5,    8,   22,    6,    2,    4,   10,   11,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         51,   79,   79,    0,    0,    0,    0,    0,    0,    0,   44,
         81,   82,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  448,  165,  227,  262,  117,  193,
          0,  137,   66,  237,   93,   40,   39,  145,  132,   62,   44,
         28,    7,   29,    1,    1,    1,    1,    1,    1,    0,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    0,    0,    0,    0,    0,    0,   

In [123]:
# Create observation_space and action_space
action_space = environment.action_space
observation_space = environment.observation_space
game = environment.game  # TODO(nadir.sidi): Review Game class

# Create a do_nothing action (all values in action_space set to zero)
action_do_nothing = action_space.get_do_nothing_action()  # This is a vector of length 76

# Run one step to get observations
# obs, *_ = environment.step(action_do_nothing)
# print(type(obs))
# obs.shape  # This is just a vector of length 538

# Get the Observation class from the game to get the minimalist ac observations
obs_min = game.export_observation().as_ac_minimalist()
obs_min.as_array().shape

##
# Create the q_table and init with arbitrary values
# Using the minimalist observations, because the docs say this automatically trims the 
# values that remain constant
##
q_table = np.zeros([obs_min.as_array().shape[0], action_do_nothing.shape[0]])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1, 100001):
    environment.reset()  # Initialize to a random state
    obs_min = game.export_observation().as_ac_minimalist().as_array()
    state = obs_min.astype(int)  # A bit of a hack converting values to ints for q-learning indices
    
    # Create observation_space and action_space
    action_space = environment.action_space
    observation_space = environment.observation_space
    game = environment.game  # TODO(nadir.sidi): Review Game class

    # Init Vars
    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        
        if random.uniform(0, 1) < epsilon:
            # Do something random learn from the action space
            # Init a blank action
            action = action_space.get_do_nothing_action(as_class_Action=True)
            # Randomly switch lines
            action_space.set_lines_status_switch_from_id(action=action,
                                                        line_id = np.random.randint(
                                                        action_space.lines_status_subaction_length),
                                                        new_switch_value=1)
            # Randomly switch substations
            # TODO(nadir.sidi): Figure out what methods to use in action_space to do this
            
        else:
            # Check the learned values to choose an action
            action = np.argmax(q_table[state])

        ##    
        # Take an action, and we want to get the new state,
        # the current reward, and whether we've hit any stopping criteria
        ##
        environment.step(action)
        
        # Get the new state
        obs_min = game.export_observation().as_ac_minimalist()
        state = obs_min.as_array()
        
        # Get the reward
        # TODO(nadir.sidi): How do you get the reward?
        # Placeholder for now; Should be higher if line capacity usage is lower
        reward = 100 - game.export_observation().get_lines_capacity_usage().sum()
        
        # Figure out if we've reached a game-over
        producers_cut = game.n_prods_cut
        if producers_cut > 0:
            done = True

        # Get the old q-value
        old_value = q_table[state, action]  
        
        # Look-up the next maximum q action given the new state
        next_max = np.max(q_table[next_state])

        # Update the new q value
        new_value = (1 - alpha) * old_value + alpha * \
            (reward + gamma * next_max)
        q_table[state, action] = new_value

#         if reward == -10:
#             penalties += 1

        state = next_state  # Not sure we need to do this, I think environment handles it?
        epochs += 1

    # Mark an "episode" every 100 epochs (?)    
    if i % 100 == 0:
        clear_output(wait=True)
        print("Episode: {i}")

print("Training finished.\n")

IndexError: index 2018 is out of bounds for axis 0 with size 392

In [51]:
environment.reset()


array([ 1.80000000e+01,  8.05999985e+01,  3.99000015e+01,  6.19999981e+00,
        9.10000038e+00,  2.40000000e+01,  7.19999981e+00,  2.79999995e+00,
        5.09999990e+00,  1.11999998e+01,  1.26000004e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.77999992e+01,  7.91999969e+01,
        3.85999985e+01,  6.19999981e+00,  9.00000000e+00,  2.33999996e+01,
        7.19999981e+00,  2.79999995e+00,  5.00000000e+00,  1.11000004e+01,
        1.23999996e+01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.83931547e+01,  7.18000031e+01,  8.15000000e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  